# Scraping the Player Rosters
### The Goal:
Scrape the website to build a json file that holds each player name as the key. The values will be the player's dashboard url, and a list of the years they were on the team.

## Imports

In [2]:
# imports 

from bs4 import BeautifulSoup
import requests
from urllib.parse import urljoin
import json
import re


## Player Dictionary
Creating the player dictionary with BeautifulSoup

Minor AI help to figure out syntax of BeautifulSoup and html access

In [3]:
## original url for the rosters
base_url = "https://fightingillini.com/sports/mens-basketball/roster"
res = requests.get(base_url)
soup = BeautifulSoup(res.text, "html.parser")
# this is the tag with the dropdown menu for all the rosters
select_tag = soup.find("select", id="ddl_past_rosters")
options = select_tag.find_all("option") if select_tag else []

season_dict = {}
player_dict = {}
no_names = []
base_url = "https://fightingillini.com"
count = 0
# for option in rosters
for option in options:
    # ex. 2024-25
    season_name = option.get_text(strip=True)[0:7]
    # ex. /sports/mens-basketball/roster/2024-25
    relative_url = option.get("value")
    ## if i need a dictionary of the roster urls, do it here 
    
    if season_name:
        full_url = urljoin(base_url, relative_url)
    season_dict[season_name] = base_url + "/sports/mens-basketball/stats/" + season_name
    res = requests.get(full_url)
    soup = BeautifulSoup(res.text, "html.parser")    

    
    count += 1
    for li in soup.find_all("li", class_="sidearm-roster-player"):
    # Get the profile URL from the data attribute
        relative_url = li.get("data-player-url")
        if relative_url:
            full_url = urljoin(base_url, relative_url)
        else:
         full_url = None

    # Find the <a> tag that has the player's name (not "Full Bio")
        name_tag = li.select_one("h3 > a")
        if not name_tag or not name_tag.text.strip():
            no_names.append(full_url)
            continue
        if name_tag and full_url:
            player_name = name_tag.text.strip()
            # I needed to check if theres already a year, cause then I'll just append instead of add a new person
            # remove this once the new year comes out
            if (player_name in player_dict) and (len(player_dict[player_name][1]) > 0):
                if '26' not in season_name:
                    player_dict[player_name][1].append(season_name)
            else:
                if '26' not in season_name:
                    player_dict[player_name] = [full_url, [season_name]]


            
    


## Product
Displaying the Player Dictionary

In [5]:
player_dict['Coleman Hawkins']

['https://fightingillini.com/sports/mens-basketball/roster/coleman-hawkins/13955',
 ['2023-24', '2022-23', '2021-22', '2020-21']]

## Send to file
creating a json file out of the dictionary so that it can be iterated through and used for the Player Dashboard Page on the app

In [ ]:
import os
output_dir = "data/processed"

os.makedirs(output_dir, exist_ok=True)
file_path = os.path.join(output_dir, 'player_list.json')
with open(file_path, 'w') as f:
    json.dump(player_dict, f, indent=4)